In [3]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import ipywidgets as widgets
import cv2
import numpy as np
import os
print("hello noobs")
print('axel was here')


hello noobs
axel was here


In [4]:
gestures = ["open_palm",
             "open_dorsal",
             "fist_palm",
             "fist_dorsal",
             "three_fingers_palm",
             "three_fingers_dorsal"]

Metoden videosToFrames behöver köras endast en gång, då loopar den genom alla id's och sparar ner alla videos som frames i följande mappstruktur: 

/hand-classifier
    /img
        /open_palm
            /102
                0.jpg
                1.jpg osv...

Alla bilder tar mycket plats tillsammans dock, landar någonstans kring 3-4 GB så vet inte riktigt hur vi gör med det.

In [ ]:
def videosToFrames():
    img_path = 'img'
    video_path = 'videos'
    dataset = pd.read_csv('video_data.csv')
    ids = ['102', '159', '294', '441', '564', '576', '609', '666', '711', '723']
    n=0
    
    for current_id in ids:
        the_path = str(video_path + '/' + current_id)
        print('Current id: ' + str(current_id))
        for video_name in gestures:
            complete_video_path = str(the_path + '/' + video_name + '.webm')
            cap = cv2.VideoCapture(complete_video_path)
            i=0
            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret == False:
                    break
                cv2.imwrite(img_path + '/' + video_name + '/' + current_id + '/' + str(i)+'.png', frame)
                i+=1
                
            cap.release()
            cv2.destroyAllWindows()
            print('video: ' + video_name + ' is done')
            
            
# Kör metoden
videosToFrames()